In [1]:
pip install langchain_openai

In [2]:
pip install langchain_community

In [3]:
pip install docarray

In [4]:
from langchain_openai.chat_models import ChatOpenAI
OPENAI_API_KEY = ""
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [5]:
from langchain.prompts import ChatPromptTemplate

template = """
Given the crime data and corresponding category and sub_category in the user context {context}, your task is to select the most relevant category from the provided {data} based on the crimedata. Provide only the selected category as the output. Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [6]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/cyberguard_hackathon/train.csv")
#df1=df[["title","text","Target_Title"]]
# df1=df.rename(columns={"Target_Title":"Book Title"})
df.head()


,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...


In [7]:
# Check for missing values in each column
print(df.isnull().sum())

category                 0
sub_category          6591
crimeaditionalinfo      21
dtype: int64


In [8]:
# Drop rows with any missing values.
df.dropna(inplace=True)

In [9]:
# Check for missing values in each column
print(df.isnull().sum())

category              0
sub_category          0
crimeaditionalinfo    0
dtype: int64


In [10]:
# prompt: Using dataframe df: do the preprocess on the crimeaditionalinfo column

import re

# Function to preprocess the text
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Handle non-string values

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply the preprocessing function to the 'crimeaditionalinfo' column
df['crimeaditionalinfo'] = df['crimeaditionalinfo'].apply(preprocess_text)

In [11]:
df.columns

Index(['category', 'sub_category', 'crimeaditionalinfo'], dtype='object')

In [12]:
pip install langchain_community

In [13]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/drive/MyDrive/cyberguard_hackathon/train.csv", csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['category', 'sub_category', 'crimeaditionalinfo']
})
#loader = CSVLoader("/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train")
text_documents = loader.load()
text_documents[:10]

[Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 0}, page_content='category: category\nsub_category: sub_category\ncrimeaditionalinfo: crimeaditionalinfo'),
 Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 1}, page_content='category: Online and Social Media Related Crime\nsub_category: Cyber Bullying  Stalking  Sexting\ncrimeaditionalinfo: I had continue received random calls and abusive messages in my whatsapp Someone added my number in a unknown facebook group name with  Only Girls  and still getting calls from unknown numbers pls help me and sort out the issue  as soon as possible Thank you'),
 Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 2}, page_content='category: Online Financial Fraud\nsub_category: Fraud CallVishing\ncrimeaditionalinfo: The above fraudster is continuously messaging me and Asking me to pay him money or he will send fake  cropp

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [15]:
pip install docarray

In [16]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Test data

In [17]:
import pandas as pd
df_test=pd.read_csv("/content/drive/MyDrive/cyberguard_hackathon/160_test_data.csv")

In [18]:
df_test1=df_test.drop(columns=["category"])

In [19]:
df_test1.head(20)

,sub_category,crimeaditionalinfo
0,NaN,Sir namaskar mein Ranjit Kumar PatraPaise neh...
1,DebitCredit Card FraudSim Swap Fraud,KOTAK MAHINDRA BANK FRAUD\nFRAUD AMOUNT
2,SQL Injection,The issue actually started when I got this ema...
3,Fraud CallVishing,I am amit kumar from karwi chitrakoot I am tot...
4,Other,I have ordered saree and blouse from rinki s...
5,Internet Banking Related Fraud,My salary of amount has to be credited to my ...
6,Unauthorised AccessData Breach,Due to some financial crisis I have applied lo...
7,UPI Related Frauds,a person called and told that i won a gift car...
8,Internet Banking Related Fraud,NameManik Varban SO Nirmal Varban \nAddkhara r...
9,DebitCredit Card FraudSim Swap Fraud,Dear Sir\n Please stop the fraudul...


In [20]:
data=list(df_test["category"][:10])

In [21]:
data

['RapeGang Rape RGRSexually Abusive Content',
 'Online Financial Fraud',
 'Cyber Attack/ Dependent Crimes',
 'Online Financial Fraud',
 'Any Other Cyber Crime',
 'Online Financial Fraud',
 'Hacking  Damage to computercomputer system etc',
 'Online Financial Fraud',
 'Online Financial Fraud',
 'Online Financial Fraud']

In [22]:
str(df_test1.iloc[1])

'sub_category             DebitCredit Card FraudSim Swap Fraud\ncrimeaditionalinfo    KOTAK MAHINDRA BANK FRAUD\\nFRAUD AMOUNT\nName: 1, dtype: object'

In [23]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

parser = StrOutputParser()
retriever=vectorstore2.as_retriever()
import random

def generate_label_subsets(labels, num_instances, num_incorrect_labels):
    """
    Generate label subsets for each instance.

    Args:
    - labels (list): List of labels.
    - num_instances (int): Number of instances.
    - num_incorrect_labels (int): Number of incorrect labels per instance.

    Returns:
    - list: List of lists, each containing the correct label followed by a list of incorrect labels.
    """
    label_subsets = []

    for _ in range(num_instances):
        correct_label = random.choice(labels)
        incorrect_labels = random.sample([label for label in labels if label != correct_label], num_incorrect_labels)
        subset = [correct_label] + incorrect_labels
        random.shuffle(subset)  # Shuffle the subset
        label_subsets.append(subset)

    return label_subsets

# Example usage
labels =data
num_instances = len(data)
num_incorrect_labels = 3


def create_list(*elements):
    """
    Create a list from the given elements.

    Args:
        *elements: Variable number of elements to be added to the list.

    Returns:
        List containing the input elements.
    """
    return list(elements)


label_subsets = generate_label_subsets(labels, num_instances, num_incorrect_labels)
for i in range(0,10):
  chain = (
    #{"context":retriever,"data":lambda x : ["Paradoxical Commandments","Creative Thoughts About A Challenging Life","A Cozy Read filled with Wisdom","Tap into Secret Wealth"],"question": RunnablePassthrough() }
     {"context":retriever,"data":lambda x :label_subsets[i],"question": RunnablePassthrough() }
    # {"context":retriever,"data":{"text1":itemgetter("text1"),"text2":itemgetter("text2"),"text3":itemgetter("text3"),"text4":itemgetter("text4")}|RunnableLambda(create_list),"question": itemgetter("question") }
    #  {"context":retriever,"data":{"text1":"Paradoxical Commandments","text2":"Creative Thoughts About A Challenging Life","text3": "A Cozy Read filled with Wisdom","text4":"Tap into Secret Wealth"}|RunnableLambda(create_list),"question": RunnablePassthrough() }
    | prompt
    | model
    | parser )
for i in range(10):
  print (f"Prediction{i}.{chain.invoke(str(df_test1.iloc[i]))}")
  print (f" Actual{i}.{df_test['category'][i]}")

Prediction0.Online Financial Fraud
 Actual0.RapeGang Rape RGRSexually Abusive Content
Prediction1.Online Financial Fraud
 Actual1.Online Financial Fraud
Prediction2.Cyber Attack/ Dependent Crimes
 Actual2.Cyber Attack/ Dependent Crimes
Prediction3.Online Financial Fraud
 Actual3.Online Financial Fraud
Prediction4.Any Other Cyber Crime
 Actual4.Any Other Cyber Crime
Prediction5.Online Financial Fraud
 Actual5.Online Financial Fraud
Prediction6.Hacking  Damage to computercomputer system etc
 Actual6.Hacking  Damage to computercomputer system etc
Prediction7.Online Financial Fraud
 Actual7.Online Financial Fraud
Prediction8.Online Financial Fraud
 Actual8.Online Financial Fraud
Prediction9.Online Financial Fraud
 Actual9.Online Financial Fraud


Testing with single datapoint

In [28]:
# Define a single test data point
single_data_point = {
    "text": "my laptop is broken into pieces",  # Replace this with the actual text you want to test
    "category": "fraud"  # Replace this with the expected category, if available
}

# Create a subset of labels for this single data point
single_label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

# Run the chain on this single data point
single_chain = (
    {"context": retriever, "data": lambda x: single_label_subset, "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

# Invoke the chain for the single data point
prediction = single_chain.invoke(str(single_data_point["text"]))
print(f"Prediction: {prediction}")
print(f"Actual: {single_data_point['category']}")

Prediction: Any Other Cyber Crime
Actual: fraud


Finding only the accuracy

In [25]:
import pandas as pd

# Load the CSV file
df_test = pd.read_csv("/content/10test.csv")  # Replace "test_data.csv" with your actual CSV file path

# Initialize a counter for correct predictions
correct_predictions = 0

# Define the number of incorrect labels for each instance
num_incorrect_labels = 3

# Loop through each row in the DataFrame
for index, row in df_test.iterrows():
    text = row["category"]  # Replace with the actual text column name
    actual_category = row["category"]  # Replace with the actual category column name

    # Generate a label subset for this instance
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create and run the chain
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt
        | model
        | parser
    )

    # Get the prediction
    prediction = single_chain.invoke(str(text)).strip()

    # Check if the prediction matches the actual category
    if prediction.lower() == actual_category.strip().lower():
        correct_predictions += 1  # Increment correct count if prediction matches

    print(f"Row {index+1} - Prediction: {prediction}, Actual: {actual_category}, Correct: {prediction.lower() == actual_category.strip().lower()}")

# Calculate accuracy
total_predictions = len(df_test)
accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy: {accuracy}%")


Row 1 - Prediction: RapeGang Rape RGRSexually Abusive Content, Actual: RapeGang Rape RGRSexually Abusive Content, Correct: True
Row 2 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 3 - Prediction: Hacking  Damage to computercomputer system etc, Actual: Cyber Attack/ Dependent Crimes, Correct: False
Row 4 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 5 - Prediction: Any Other Cyber Crime, Actual: Any Other Cyber Crime, Correct: True
Row 6 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 7 - Prediction: Cyber Attack/ Dependent Crimes, Actual: Hacking  Damage to computercomputer system etc, Correct: False
Row 8 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 9 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 10 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: Tr

Finding the
Accuracy,

Precision,

Recall,

F1 score and

Confusion matrix

In [26]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

# Load the CSV file
df_test = pd.read_csv("/content/10test.csv")  # Replace "/content/10test.csv" with your actual CSV file path

# List of possible labels (update this with your actual labels)
# labels = [...]  # Populate with your actual labels
labels = df['category'].values

# Define the number of incorrect labels for each instance
num_incorrect_labels = 3

# Initialize lists to store true and predicted labels
y_true = []
y_pred = []

# Loop through each row in the DataFrame
for index, row in df_test.iterrows():
    text = row["category"]  # Replace with the actual text column name
    actual_category = row["category"]  # Replace with the actual category column name
    y_true.append(actual_category)  # Append the actual category to y_true

    # Generate a label subset for this instance
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create and run the chain
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt
        | model
        | parser
    )

    # Get the prediction
    try:
        prediction = single_chain.invoke(str(text)).strip()
    except Exception as e:
        print(f"Error in prediction for row {index+1}: {e}")
        prediction = ""

    y_pred.append(prediction)  # Append the prediction to y_pred

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')  # Adjust `average` if needed
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

# Display results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

# For ROC-AUC (only for binary classification cases)
if len(set(y_true)) == 2:
    # y_pred_proba would need to be generated by the model, representing probability of each class
    # Assuming y_pred_proba is available
    roc_auc = roc_auc_score(y_true, y_pred_proba[:, 1])  # Assuming `y_pred_proba` is the probability of positive class
    show(f"ROC-AUC Score: {roc_auc}")



Accuracy: 0.7222222222222222
Precision: 0.6753246753246754
Recall: 0.7222222222222222
F1 Score: 0.6658068783068782
Confusion Matrix:
[[ 4  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  0  0  0  0  0]
 [ 1  0  0  0  1  0  0  0  0]
 [ 0  0  0  0 19  0  0  0  0]
 [ 1  0  0  0  1  2  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Building the model for Sub_category as targetted variable

In [29]:
from langchain.prompts import ChatPromptTemplate

template1 = """
Given the crime data and corresponding category and sub_category in the user context {context}, your task is to select the most relevant sub_category from the provided {data} based on the crimedata. Provide only the selected sub_category as the output. Context: {context}

Question: {question}
"""

prompt1 = ChatPromptTemplate.from_template(template1)

In [30]:
import pandas as pd
dfs=pd.read_csv("/content/drive/MyDrive/cyberguard_hackathon/train.csv")
#df1=df[["title","text","Target_Title"]]
# df1=df.rename(columns={"Target_Title":"Book Title"})
dfs.head()

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...


In [31]:
# Check for missing values in each column
print(dfs.isnull().sum())

category                 0
sub_category          6591
crimeaditionalinfo      21
dtype: int64


In [32]:
dfs.dropna(inplace=True)

In [33]:
# Check for missing values in each column
print(dfs.isnull().sum())

category              0
sub_category          0
crimeaditionalinfo    0
dtype: int64


Preprocessing the text column

In [34]:
# prompt: Using dataframe df: do the preprocess on the crimeaditionalinfo column

import re

# Function to preprocess the text
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Handle non-string values

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply the preprocessing function to the 'crimeaditionalinfo' column
dfs['crimeaditionalinfo'] = dfs['crimeaditionalinfo'].apply(preprocess_text)


In [35]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/drive/MyDrive/cyberguard_hackathon/train.csv", csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['category', 'sub_category', 'crimeaditionalinfo']
})
#loader = CSVLoader("/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train")
text_documents1 = loader.load()
text_documents1[:10]

[Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 0}, page_content='category: category\nsub_category: sub_category\ncrimeaditionalinfo: crimeaditionalinfo'),
 Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 1}, page_content='category: Online and Social Media Related Crime\nsub_category: Cyber Bullying  Stalking  Sexting\ncrimeaditionalinfo: I had continue received random calls and abusive messages in my whatsapp Someone added my number in a unknown facebook group name with  Only Girls  and still getting calls from unknown numbers pls help me and sort out the issue  as soon as possible Thank you'),
 Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 2}, page_content='category: Online Financial Fraud\nsub_category: Fraud CallVishing\ncrimeaditionalinfo: The above fraudster is continuously messaging me and Asking me to pay him money or he will send fake  cropp

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter1 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents1 = text_splitter1.split_documents(text_documents1)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore1 = DocArrayInMemorySearch.from_documents(documents1, embeddings)

In [ ]:
import pandas as pd
dfs_test=pd.read_csv("/content/drive/MyDrive/cyberguard_hackathon/160_test_data.csv")

In [ ]:
dfs_test1=dfs_test.drop(columns=["sub_category"])

In [ ]:
dfs_test1.isnull().sum()

In [ ]:
dfs_test.dropna(inplace = True)

In [ ]:
datas=list(dfs_test["sub_category"][:10])

In [ ]:
str(dfs_test1.iloc[1])

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

parser = StrOutputParser()
retriever=vectorstore1.as_retriever()
import random

def generate_label_subsets(labels, num_instances, num_incorrect_labels):
    """
    Generate label subsets for each instance.

    Args:
    - labels (list): List of labels.
    - num_instances (int): Number of instances.
    - num_incorrect_labels (int): Number of incorrect labels per instance.

    Returns:
    - list: List of lists, each containing the correct label followed by a list of incorrect labels.
    """
    label_subsets = []

    for _ in range(num_instances):
        correct_label = random.choice(labels)
        incorrect_labels = random.sample([label for label in labels if label != correct_label], num_incorrect_labels)
        subset = [correct_label] + incorrect_labels
        random.shuffle(subset)  # Shuffle the subset
        label_subsets.append(subset)

    return label_subsets

# Example usage
labels =datas
num_instances = len(datas)
num_incorrect_labels = 3


def create_list(*elements):
    """
    Create a list from the given elements.

    Args:
        *elements: Variable number of elements to be added to the list.

    Returns:
        List containing the input elements.
    """
    return list(elements)


label_subsets = generate_label_subsets(labels, num_instances, num_incorrect_labels)

# for i in range(0,10):
#   chain = (
#     #{"context":retriever,"data":lambda x : ["Paradoxical Commandments","Creative Thoughts About A Challenging Life","A Cozy Read filled with Wisdom","Tap into Secret Wealth"],"question": RunnablePassthrough() }
#      {"context":retriever,"data":lambda x :label_subsets[i],"question": RunnablePassthrough() }
#     # {"context":retriever,"data":{"text1":itemgetter("text1"),"text2":itemgetter("text2"),"text3":itemgetter("text3"),"text4":itemgetter("text4")}|RunnableLambda(create_list),"question": itemgetter("question") }
#     #  {"context":retriever,"data":{"text1":"Paradoxical Commandments","text2":"Creative Thoughts About A Challenging Life","text3": "A Cozy Read filled with Wisdom","text4":"Tap into Secret Wealth"}|RunnableLambda(create_list),"question": RunnablePassthrough() }
#     | prompt1
#     | model
#     | parser )
# for i in range(10):
#   print (f"Prediction{i}.{chain.invoke(str(dfs_test1.iloc[i]))}")
#   print (f" Actual{i}.{dfs_test['sub_category'][i]}")


for i in range(10):
  # Get the text for prediction and actual category for the i-th row
  input_text = dfs_test1.iloc[i]  # Access the i-th row for prediction input
  actual_category = dfs_test.iloc[i]['sub_category']  # Access the category column for actual value

  # Run the prediction chain
  prediction = chain.invoke(str(input_text))

  # Display the results
  print(f"Prediction {i}: {prediction}")
  print(f"Actual {i}: {actual_category}")


Calculating Accuracy, Precision, Recall, F1 score and confusion matrix

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

# Load the CSV file
dfs_test = pd.read_csv("/content/10test.csv")  # Replace with your actual CSV file path

# List of possible labels (assuming they are in `dfs['sub_category']`)
labels = dfs['sub_category'].values

# Define the number of incorrect labels for each instance
num_incorrect_labels = 3

# Initialize lists to store true and predicted labels
y_true = []
y_pred = []

# Loop through each row in the DataFrame
for index, row in dfs_test.iterrows():
    text = row["sub_category"]  # Replace with the actual text column name
    actual_category = row["sub_category"]  # Replace with the actual category column name
    y_true.append(actual_category)  # Append the actual category to y_true

    # Generate a label subset for this instance
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create and run the chain with the correct variable name for "data"
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt1
        | model
        | parser
    )

    # Get the prediction
    try:
        prediction = single_chain.invoke(str(text)).strip()
    except Exception as e:
        print(f"Error in prediction for row {index+1}: {e}")
        prediction = ""

    y_pred.append(prediction)  # Append the prediction to y_pred

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')  # Adjust `average` if needed
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

# Display results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

# For ROC-AUC (only for binary classification cases)
if len(set(y_true)) == 2:
    # Ensure `y_pred_proba` exists and represents probability of each class
    # Assuming `y_pred_proba` contains probabilities of the positive class
    roc_auc = roc_auc_score(y_true, y_pred_proba[:, 1])
    print(f"ROC-AUC Score: {roc_auc}")


In [ ]:
import pandas as pd
df1_test=pd.read_csv("/content/10test.csv")

In [ ]:
df1_test1=df1_test.drop(columns=["category"])

In [ ]:
df1_test['category'].value_counts()

,count
category,
Online Financial Fraud,19
Any Other Cyber Crime,4
Online and Social Media Related Crime,4
Cyber Attack/ Dependent Crimes,3
Hacking Damage to computercomputer system etc,2
RapeGang Rape RGRSexually Abusive Content,1
Sexually Explicit Act,1
Sexually Obscene material,1
Child Pornography CPChild Sexual Abuse Material CSAM,1


In [ ]:
df1_test1.dropna(inplace = True)

In [ ]:
df1_test1.head(20)

,sub_category,crimeaditionalinfo
1,DebitCredit Card FraudSim Swap Fraud,Sir namaskar mein Ranjit Kumar PatraPaise ne...
2,SQL Injection,The issue actually started when I got this ema...
3,Fraud CallVishing,I am amit kumar from karwi chitrakoot I am tot...
4,Other,I have ordered saree and blouse from rinki s...
5,Internet Banking Related Fraud,My salary of amount has to be credited to my ...
6,Unauthorised AccessData Breach,Due to some financial crisis I have applied lo...
7,UPI Related Frauds,a person called and told that i won a gift car...
8,Internet Banking Related Fraud,NameManik Varban SO Nirmal Varban \nAddkhara r...
9,DebitCredit Card FraudSim Swap Fraud,Dear Sir\n Please stop the fraudul...
11,Damage to computer computer systems etc,PLEASE READ ABOVE ATTACHED COMPLAINT OVERVIEW\...


In [ ]:
df1_test1.crimeaditionalinfo[2]

"The issue actually started when I got this email, which at first glance seemed like spam. I usually ignore such things, but for some reason, I opened it this time. Afterward, my phone started freezing, and it wasn’t normal. I thought it was a technical glitch, but now I’m not so sure. I don't know how, but it’s like they knew everything I was doing, even though I was trying to be careful. It’s terrifying to think about. The more I tried to fix it, the more problems came up. My work account was hackd, and then I got locked out of everything. This has really left me anxious and frustrated, I can’t sleep knowing my information is out there. I’ve even considered deleting all my accounts, but that’s not a solution. Even my phone isn’t working right anymore. It’s so stange how everything just fell apart after that one email."

In [ ]:
data1=list(df1_test["category"][:10])

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

parser = StrOutputParser()
retriever=vectorstore2.as_retriever()
import random

def generate_label_subsets(labels, num_instances, num_incorrect_labels):
    """
    Generate label subsets for each instance.

    Args:
    - labels (list): List of labels.
    - num_instances (int): Number of instances.
    - num_incorrect_labels (int): Number of incorrect labels per instance.

    Returns:
    - list: List of lists, each containing the correct label followed by a list of incorrect labels.
    """
    label_subsets = []

    for _ in range(num_instances):
        correct_label = random.choice(labels)
        incorrect_labels = random.sample([label for label in labels if label != correct_label], num_incorrect_labels)
        subset = [correct_label] + incorrect_labels
        random.shuffle(subset)  # Shuffle the subset
        label_subsets.append(subset)

    return label_subsets

# Example usage
labels =data1
num_instances = len(data1)
num_incorrect_labels = 3


def create_list(*elements):
    """
    Create a list from the given elements.

    Args:
        *elements: Variable number of elements to be added to the list.

    Returns:
        List containing the input elements.
    """
    return list(elements)


label_subsets = generate_label_subsets(labels, num_instances, num_incorrect_labels)
for i in range(0,10):
  chain = (
    #{"context":retriever,"data":lambda x : ["Paradoxical Commandments","Creative Thoughts About A Challenging Life","A Cozy Read filled with Wisdom","Tap into Secret Wealth"],"question": RunnablePassthrough() }
     {"context":retriever,"data":lambda x :label_subsets[i],"question": RunnablePassthrough() }
    # {"context":retriever,"data":{"text1":itemgetter("text1"),"text2":itemgetter("text2"),"text3":itemgetter("text3"),"text4":itemgetter("text4")}|RunnableLambda(create_list),"question": itemgetter("question") }
    #  {"context":retriever,"data":{"text1":"Paradoxical Commandments","text2":"Creative Thoughts About A Challenging Life","text3": "A Cozy Read filled with Wisdom","text4":"Tap into Secret Wealth"}|RunnableLambda(create_list),"question": RunnablePassthrough() }
    | prompt
    | model
    | parser )
for i in range(10):
  print (f"Prediction{i}.{chain.invoke(str(df1_test1.iloc[i]))}")
  print (f" Actual{i}.{df1_test['category'][i]}")

Prediction0.Online Financial Fraud
 Actual0.RapeGang Rape RGRSexually Abusive Content
Prediction1.Cyber Attack/ Dependent Crimes
 Actual1.Online Financial Fraud
Prediction2.Online Financial Fraud
 Actual2.Cyber Attack/ Dependent Crimes
Prediction3.Any Other Cyber Crime
 Actual3.Online Financial Fraud
Prediction4.Online Financial Fraud
 Actual4.Any Other Cyber Crime
Prediction5.Hacking  Damage to computercomputer system etc
 Actual5.Online Financial Fraud
Prediction6.Online Financial Fraud
 Actual6.Hacking  Damage to computercomputer system etc
Prediction7.Online Financial Fraud
 Actual7.Online Financial Fraud
Prediction8.Online Financial Fraud
 Actual8.Online Financial Fraud
Prediction9.Hacking  Damage to computercomputer system etc
 Actual9.Online Financial Fraud


In [ ]:
df['category'].value_counts()

,count
category,
Online Financial Fraud,57416
Online and Social Media Related Crime,12138
Any Other Cyber Crime,10877
Cyber Attack/ Dependent Crimes,3608
Hacking Damage to computercomputer system etc,1710
Cryptocurrency Crime,480
Online Gambling Betting,444
Online Cyber Trafficking,183
Cyber Terrorism,161


Testing with Single Datapoint

Prediction: Any Other Cyber Crime
Actual: fraud


Testing with single datapoint

In [ ]:
# Define the test data points and their actual categories
test_data_points = [
    {"text": "Sample text 1", "category": "Expected category 1"},
    {"text": "Sample text 2", "category": "Expected category 2"},
    # Add more test data points as needed
]

# Initialize a counter for correct predictions
correct_predictions = 0

# Iterate over each test data point
for i, data_point in enumerate(test_data_points):
    # Generate label subset for the current data point
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create a chain for the current data point
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt
        | model
        | parser
    )

    # Run the chain and get the prediction
    prediction = single_chain.invoke(str(data_point["text"]))

    # Compare prediction with the actual category
    is_correct = prediction.strip().lower() == data_point["category"].strip().lower()
    if is_correct:
        correct_predictions += 1  # Increment if prediction is correct

    print(f"Test Data {i+1} - Prediction: {prediction}, Actual: {data_point['category']}, Correct: {is_correct}")

# Calculate accuracy
total_predictions = len(df_test)
accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy: {accuracy}%")


Finding the **accuracy**

In [ ]:
import pandas as pd

# Load the CSV file
df_test = pd.read_csv("/content/10test.csv")  # Replace "test_data.csv" with your actual CSV file path

# Initialize a counter for correct predictions
correct_predictions = 0

# Define the number of incorrect labels for each instance
num_incorrect_labels = 3

# Loop through each row in the DataFrame
for index, row in df_test.iterrows():
    text = row["category"]  # Replace with the actual text column name
    actual_category = row["category"]  # Replace with the actual category column name

    # Generate a label subset for this instance
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create and run the chain
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt
        | model
        | parser
    )

    # Get the prediction
    prediction = single_chain.invoke(str(text)).strip()

    # Check if the prediction matches the actual category
    if prediction.lower() == actual_category.strip().lower():
        correct_predictions += 1  # Increment correct count if prediction matches

    print(f"Row {index+1} - Prediction: {prediction}, Actual: {actual_category}, Correct: {prediction.lower() == actual_category.strip().lower()}")

# Calculate accuracy
total_predictions = len(df_test)
accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy: {accuracy}%")


Row 1 - Prediction: None, Actual: RapeGang Rape RGRSexually Abusive Content, Correct: False
Row 2 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 3 - Prediction: Cyber Attack/ Dependent Crimes, Actual: Cyber Attack/ Dependent Crimes, Correct: True
Row 4 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 5 - Prediction: Any Other Cyber Crime, Actual: Any Other Cyber Crime, Correct: True
Row 6 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 7 - Prediction: Hacking  Damage to computercomputer system etc, Actual: Hacking  Damage to computercomputer system etc, Correct: True
Row 8 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 9 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 10 - Prediction: Online Financial Fraud, Actual: Online Financial Fraud, Correct: True
Row 11 - Prediction: RapeGang Rape 

In [ ]:
labels = df['category'].values

calculating accuracy, precision, recall, confusion matrix,classification report

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

# Load the CSV file
df_test = pd.read_csv("/content/10test.csv")  # Replace "/content/10test.csv" with your actual CSV file path

# List of possible labels (update this with your actual labels)
# labels = [...]  # Populate with your actual labels
labels = df['category'].values

# Define the number of incorrect labels for each instance
num_incorrect_labels = 3

# Initialize lists to store true and predicted labels
y_true = []
y_pred = []

# Loop through each row in the DataFrame
for index, row in df_test.iterrows():
    text = row["category"]  # Replace with the actual text column name
    actual_category = row["category"]  # Replace with the actual category column name
    y_true.append(actual_category)  # Append the actual category to y_true

    # Generate a label subset for this instance
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create and run the chain
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt
        | model
        | parser
    )

    # Get the prediction
    try:
        prediction = single_chain.invoke(str(text)).strip()
    except Exception as e:
        print(f"Error in prediction for row {index+1}: {e}")
        prediction = ""

    y_pred.append(prediction)  # Append the prediction to y_pred

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')  # Adjust `average` if needed
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

# Display results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

# For ROC-AUC (only for binary classification cases)
if len(set(y_true)) == 2:
    # y_pred_proba would need to be generated by the model, representing probability of each class
    # Assuming y_pred_proba is available
    roc_auc = roc_auc_score(y_true, y_pred_proba[:, 1])  # Assuming `y_pred_proba` is the probability of positive class
    show(f"ROC-AUC Score: {roc_auc}")



Accuracy: 0.7222222222222222
Precision: 0.6087962962962963
Recall: 0.7222222222222222
F1 Score: 0.6507529507529507
Confusion Matrix:
[[ 3  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0]
 [ 3  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 19  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Predicting the sub-category

In [5]:
from langchain.prompts import ChatPromptTemplate

template1 = """
Given the crime data and corresponding category and sub_category in the user context {context}, your task is to select the most relevant sub_category from the provided {data} based on the crimedata. Provide only the selected sub_category as the output. Context: {context}

Question: {question}
"""

prompt1 = ChatPromptTemplate.from_template(template1)

In [6]:
import pandas as pd
dfs=pd.read_csv("/content/drive/MyDrive/cyberguard_hackathon/train.csv")
#df1=df[["title","text","Target_Title"]]
# df1=df.rename(columns={"Target_Title":"Book Title"})
dfs.head()

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...


In [7]:
# Check for missing values in each column
print(dfs.isnull().sum())

category                 0
sub_category          6591
crimeaditionalinfo      21
dtype: int64


In [8]:
dfs.dropna(inplace=True)

In [9]:
# Check for missing values in each column
print(dfs.isnull().sum())

category              0
sub_category          0
crimeaditionalinfo    0
dtype: int64


Preprocessing the text column

In [10]:
# prompt: Using dataframe df: do the preprocess on the crimeaditionalinfo column

import re

# Function to preprocess the text
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Handle non-string values

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply the preprocessing function to the 'crimeaditionalinfo' column
dfs['crimeaditionalinfo'] = dfs['crimeaditionalinfo'].apply(preprocess_text)

In [11]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/drive/MyDrive/cyberguard_hackathon/train.csv", csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['category', 'sub_category', 'crimeaditionalinfo']
})
#loader = CSVLoader("/content/drive/MyDrive/Colab Notebooks/RAG/filtered_train")
text_documents1 = loader.load()
text_documents1[:10]

[Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 0}, page_content='category: category\nsub_category: sub_category\ncrimeaditionalinfo: crimeaditionalinfo'),
 Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 1}, page_content='category: Online and Social Media Related Crime\nsub_category: Cyber Bullying  Stalking  Sexting\ncrimeaditionalinfo: I had continue received random calls and abusive messages in my whatsapp Someone added my number in a unknown facebook group name with  Only Girls  and still getting calls from unknown numbers pls help me and sort out the issue  as soon as possible Thank you'),
 Document(metadata={'source': '/content/drive/MyDrive/cyberguard_hackathon/train.csv', 'row': 2}, page_content='category: Online Financial Fraud\nsub_category: Fraud CallVishing\ncrimeaditionalinfo: The above fraudster is continuously messaging me and Asking me to pay him money or he will send fake  cropp

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter1 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents1 = text_splitter1.split_documents(text_documents1)

In [13]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore1 = DocArrayInMemorySearch.from_documents(documents1, embeddings)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [14]:
import pandas as pd
dfs_test=pd.read_csv("/content/drive/MyDrive/cyberguard_hackathon/160_test_data.csv")

In [15]:
dfs_test1=dfs_test.drop(columns=["sub_category"])

In [16]:
dfs_test1.isnull().sum()

,0
category,0
crimeaditionalinfo,0


In [17]:
dfs_test.dropna(inplace = True)

In [18]:
datas=list(dfs_test["sub_category"][:10])

In [19]:
str(dfs_test1.iloc[1])

'category                               Online Financial Fraud\ncrimeaditionalinfo    KOTAK MAHINDRA BANK FRAUD\\nFRAUD AMOUNT\nName: 1, dtype: object'

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda

parser = StrOutputParser()
retriever=vectorstore1.as_retriever()
import random

def generate_label_subsets(labels, num_instances, num_incorrect_labels):
    """
    Generate label subsets for each instance.

    Args:
    - labels (list): List of labels.
    - num_instances (int): Number of instances.
    - num_incorrect_labels (int): Number of incorrect labels per instance.

    Returns:
    - list: List of lists, each containing the correct label followed by a list of incorrect labels.
    """
    label_subsets = []

    for _ in range(num_instances):
        correct_label = random.choice(labels)
        incorrect_labels = random.sample([label for label in labels if label != correct_label], num_incorrect_labels)
        subset = [correct_label] + incorrect_labels
        random.shuffle(subset)  # Shuffle the subset
        label_subsets.append(subset)

    return label_subsets

# Example usage
labels =datas
num_instances = len(datas)
num_incorrect_labels = 3


def create_list(*elements):
    """
    Create a list from the given elements.

    Args:
        *elements: Variable number of elements to be added to the list.

    Returns:
        List containing the input elements.
    """
    return list(elements)


label_subsets = generate_label_subsets(labels, num_instances, num_incorrect_labels)

# for i in range(0,10):
#   chain = (
#     #{"context":retriever,"data":lambda x : ["Paradoxical Commandments","Creative Thoughts About A Challenging Life","A Cozy Read filled with Wisdom","Tap into Secret Wealth"],"question": RunnablePassthrough() }
#      {"context":retriever,"data":lambda x :label_subsets[i],"question": RunnablePassthrough() }
#     # {"context":retriever,"data":{"text1":itemgetter("text1"),"text2":itemgetter("text2"),"text3":itemgetter("text3"),"text4":itemgetter("text4")}|RunnableLambda(create_list),"question": itemgetter("question") }
#     #  {"context":retriever,"data":{"text1":"Paradoxical Commandments","text2":"Creative Thoughts About A Challenging Life","text3": "A Cozy Read filled with Wisdom","text4":"Tap into Secret Wealth"}|RunnableLambda(create_list),"question": RunnablePassthrough() }
#     | prompt1
#     | model
#     | parser )
# for i in range(10):
#   print (f"Prediction{i}.{chain.invoke(str(dfs_test1.iloc[i]))}")
#   print (f" Actual{i}.{dfs_test['sub_category'][i]}")


# Corrected loop where 'chain' is defined inside
for i in range(10):
    # Create a new chain for each iteration using label_subsets
    chain = (
        {"context": retriever, "data": lambda x: label_subsets[i], "question": RunnablePassthrough()}
        | prompt1
        | model
        | parser
    )

    # Get the text for prediction and actual category for the i-th row
    input_text = dfs_test1.iloc[i]  # Access the i-th row for prediction input
    actual_category = dfs_test.iloc[i]['sub_category']  # Access the category column for actual value

    # Run the prediction chain
    prediction = chain.invoke(str(input_text))

    # Display the results
    print(f"Prediction {i}: {prediction}")
    print(f"Actual {i}: {actual_category}")


Prediction 0: None
Actual 0: nan
Prediction 1: Internet Banking Related Fraud
Actual 1: DebitCredit Card FraudSim Swap Fraud
Prediction 2: Internet Banking Related Fraud
Actual 2: SQL Injection
Prediction 3: Internet Banking Related Fraud
Actual 3: Fraud CallVishing
Prediction 4: Other
Actual 4: Other
Prediction 5: Internet Banking Related Fraud
Actual 5: Internet Banking Related Fraud
Prediction 6: Damage to computer computer systems etc
Actual 6: Unauthorised AccessData Breach
Prediction 7: Fraud CallVishing
Actual 7: UPI Related Frauds
Prediction 8: Internet Banking Related Fraud
Actual 8: Internet Banking Related Fraud
Prediction 9: Internet Banking Related Fraud
Actual 9: DebitCredit Card FraudSim Swap Fraud


In [21]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

# Load the CSV file
dfs_test = pd.read_csv("/content/10test.csv")  # Replace with your actual CSV file path

# List of possible labels (assuming they are in `dfs['sub_category']`)
labels = dfs['sub_category'].values

# Define the number of incorrect labels for each instance
num_incorrect_labels = 3

# Initialize lists to store true and predicted labels
y_true = []
y_pred = []

# Loop through each row in the DataFrame
for index, row in dfs_test.iterrows():
    text = row["sub_category"]  # Replace with the actual text column name
    actual_category = row["sub_category"]  # Replace with the actual category column name
    y_true.append(actual_category)  # Append the actual category to y_true

    # Generate a label subset for this instance
    label_subset = generate_label_subsets(labels, 1, num_incorrect_labels)[0]

    # Create and run the chain with the correct variable name for "data"
    single_chain = (
        {"context": retriever, "data": lambda x: label_subset, "question": RunnablePassthrough()}
        | prompt1
        | model
        | parser
    )

    # Get the prediction
    try:
        prediction = single_chain.invoke(str(text)).strip()
    except Exception as e:
        print(f"Error in prediction for row {index+1}: {e}")
        prediction = ""

    y_pred.append(prediction)  # Append the prediction to y_pred

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')  # Adjust `average` if needed
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

# Display results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

# For ROC-AUC (only for binary classification cases)
if len(set(y_true)) == 2:
    # Ensure `y_pred_proba` exists and represents probability of each class
    # Assuming `y_pred_proba` contains probabilities of the positive class
    roc_auc = roc_auc_score(y_true, y_pred_proba[:, 1])
    print(f"ROC-AUC Score: {roc_auc}")


Accuracy: 0.4166666666666667
Precision: 0.39292328042328045
Recall: 0.4166666666666667
F1 Score: 0.38170194003527336
Confusion Matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 9 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 2 0 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
